In [42]:
import pandas as pd
import numpy as np
import requests
import json
import time
from datetime import datetime, timedelta

## Helpers

In [18]:
def get_list_of_dates(start=datetime.today() - timedelta(weeks=52), 
                      end=datetime.today(), 
                      frequency="W"):
  
  date_list = pd.date_range(start, end, freq=frequency)
  date_list_str = [x.strftime("%Y-%m-%d") for x in date_list]
  return date_list_str

In [37]:
def generate_request(list_name="fiction", date="current", nyt_api_key=None):
  """
  Generate the API request url
  """
  
  base = "https://api.nytimes.com/svc/books/v3"
  
  if list_name == "fiction":
      list_spec = f"/lists/{date}/combined-print-and-e-book-fiction.json"
  elif list_name == "non-fiction":
      list_spec = f"/lists/{date}/combined-print-and-e-book-nonfiction.json"
  else:
      list_spec = f"/lists/{date}/{list_name}.json"
  
  api_url = f"{base}{list_spec}?api-key={nyt_api_key}"
  
  return api_url

In [40]:
def api_request(list_name="fiction", date="current", nyt_api_key=None):
  """
  Extract the data from the API
  """
  
  url = generate_request(list_name=list_name, date=date, nyt_api_key=nyt_api_key)
  
  response = requests.get(url)
  json_raw = response.json()
  
  nyt_list_json = json_raw["results"]["books"]
  list_publish_date = json_raw["results"]["published_date"]
  
  df_all = pd.DataFrame().from_dict(nyt_list_json)
  
  cols_to_drop = [
      "book_image",
      "book_image_width",
      "book_image_height",
      "isbns",
      "buy_links",
  ]
  
  df = df_all.drop(columns=cols_to_drop).copy()
  df["title"] = df["title"].str.title()
  df["rank_last_week"].replace({0: np.nan}, inplace=True)
  df["list_publication_date"] = list_publish_date
  
  df_out = df[["rank", "rank_last_week", "title", "author", 
    "weeks_on_list", "list_publication_date"]].copy()
  
  return df_out

In [20]:
api_key = "zezBN6Z0bPALzf9AnFF8chlCEzmbvL7M"

## Use API 

In [23]:
url = f"https://api.nytimes.com/svc/books/v3/lists/overview.json?api-key={api_key}"

response = requests.get(url)
print(response.status_code)

data = response.json()

200


In [43]:
df = api_request(nyt_api_key=api_key)

C:\Users\Sina A.Y\AppData\Local\Temp\ipykernel_122252\3135348098.py:26: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["rank_last_week"].replace({0: np.nan}, inplace=True)


In [44]:
df

,rank,rank_last_week,title,author,weeks_on_list,list_publication_date
0,1,NaN,The Widow,John Grisham,1,2025-11-09
1,2,NaN,The Proving Ground,Michael Connelly,1,2025-11-09
2,3,NaN,The Things Gods Break,Abigail Owen,1,2025-11-09
3,4,4.0,The Secret Of Secrets,Dan Brown,7,2025-11-09
4,5,1.0,Gone Before Goodbye,Reese Witherspoon and Harlan Coben,2,2025-11-09
5,6,5.0,Alchemised,SenLinYu,5,2025-11-09
6,7,2.0,Remain,Nicholas Sparks with M. Night Shyamalan,2,2025-11-09
7,8,3.0,Mate,Ali Hazelwood,3,2025-11-09
8,9,NaN,King Sorrow,Joe Hill,1,2025-11-09
9,10,NaN,Good Spirits,B.K. Borison,1,2025-11-09
